<html>
<body>
    <h1><center>Applied Data Science Capstone -Week 3 </center></h1>
    <h2>Partial submission of the capstone project for the IBM data science course - week 3 assignment submission</h2>
    <h3>Segmenting and Clustering of Neighborhood in Toronto</h3>
    </body>
</html>

<h3> Task 1 </h3>
    <ol>
        <li> Scrape the Wikipedia page to get data for the city of Toronto
        <li> Create a Dataframe
        <li> Clean the data, to get Postal Code, Borough and Neighborhood
        <li> Clean the data, delete rows with "not assigned" Borough
        <li> Clean the data, Neighborhoods with same postal code should be combined, neighborhood separated with comma
        <li> Clean the data, Borough with "not assigned" neighborhood should be have same name for borough and neighborhood
        </ol>

In [115]:
#import pandas to the notebook
import pandas as pd
import requests as rq

In [116]:
!pip install beautifulsoup4

In [117]:
# get the wikipedia page for the data
page = rq.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# check the status of the request
print("The page status code is:" , page.status_code)

The page status code is: 200


<h5>The page status code of 200 suggests that the page has been sucessfully recieved, the next step would be to get the page structure for the wikipedia page to get the overall structure</h5> 

In [118]:
!pip install lxml

<h5>The prettify makes the report overly populated hence although I have executed the command, I will not run it in this notebook. The postal codes are in table section of the page </h5>

In [119]:
from bs4 import BeautifulSoup as BSoup
soup = BSoup(page.text, 'html.parser')
#soup  = BSoup(page, 'lxml')
# print(soup.prettify())

In [120]:
# Get the table into a list
postalCodeList = list(soup.table)
type(postalCodeList)
#postalCodeList[]

list

In [121]:
header = soup.find('tr')
colname = [col.get_text() for col in header.find_all('th')]
colname[2] = 'Neighborhood'
df = pd.DataFrame(columns=colname)
df

,Postcode,Borough,Neighborhood


In [122]:
n_rows = []
Areas  = soup.find_all('tr')
for Area in Areas:
    rows = [row.get_text() for row in Area.find_all('td')]
    n_rows.append(rows)
    temp_df = pd.DataFrame(n_rows)
temp_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,M1A,Not assigned,Not assigned\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,M2A,Not assigned,Not assigned\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,M3A,North York,Parkwoods\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,M4A,North York,Victoria Village\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,M5A,Downtown Toronto,Harbourfront\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,M6A,North York,Lawrence Heights\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,M6A,North York,Lawrence Manor\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,M7A,Downtown Toronto,Queen's Park\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,M8A,Not assigned,Not assigned\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [123]:
# As it can be observed that a lot of columns are rows are unnecessary and need to be removed from dataframe
# remove rows 0 and 288 thru 293
temp_df = temp_df.drop(temp_df.index[0])
temp_df = temp_df.drop(temp_df.index[287:293])

#rempve columns 3:32
temp_df = temp_df.drop(temp_df.columns[3:31], axis =1)
temp_df.columns = colname


#remove '\n' from dataframe cells
temp_df = temp_df.replace('\n','', regex=True) 


In [124]:
df = temp_df.reset_index()

In [125]:
# Remove all the rows with Boroughs not assigned
df = df[~df.Borough.str.contains("Not assigned")].reset_index()

In [126]:
# Assign the name of Borough where the Neighborhood is not assigned
import numpy as np
df['Neighborhood'] = np.where(df['Neighborhood'] != 'Not assigned',df['Neighborhood'], df['Borough'])
df.head(10)

,level_0,index,Postcode,Borough,Neighborhood
0,2,3,M3A,North York,Parkwoods
1,3,4,M4A,North York,Victoria Village
2,4,5,M5A,Downtown Toronto,Harbourfront
3,5,6,M6A,North York,Lawrence Heights
4,6,7,M6A,North York,Lawrence Manor
5,7,8,M7A,Downtown Toronto,Queen's Park
6,9,10,M9A,Queen's Park,Queen's Park
7,10,11,M1B,Scarborough,Rouge
8,11,12,M1B,Scarborough,Malvern
9,13,14,M3B,North York,Don Mills North


In [127]:
# Combine the neighborhood with same postal code
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.shape

(103, 3)

<h3> This is the partial submission of the peer review assignment</h3>


<h5> According to the requirements of the assignment</h5>
<ol>
    <li>Data was scraped from the beautiful soup </li>
    <li>Lot of information was dropped</li>
    <li>'Not assigned' borough rows were removed </li>
    <li>'Not assigned' neighborhoods were given borough name </li>
    <li>Neighborhoods with same postal code were combined with comma separator</li>
    <li>Finally the shape of the data was found </li>
</ol>


<h4>The next step in this process is to add the Latitude and Longitude data to Postal Code</h4>

In [128]:
# read the CSV 
latlng_df = pd.read_csv('https://cocl.us/Geospatial_data')
# Change the name of Postal Code column to Postcode
latlng_df.rename(columns = {'Postal Code':'Postcode'}, inplace=True)
# join the df and latlng_df with Postal Code
final_df = pd.merge(df, latlng_df, on='Postcode')
final_df.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [129]:
final_df.shape

(103, 5)

<h4>The final part is to add the data on map of Toronto</h4>

In [130]:
# create lists from the dataframe
latitude = final_df['Latitude'].values.tolist()
longitude = final_df['Longitude'].values.tolist()
borough = final_df['Borough'].values.tolist()

neighborhood = final_df['Neighborhood'].values.tolist()
neighborhood[10]

'Dorset Park, Scarborough Town Centre, Wexford Heights'

In [171]:
import folium
maps_toronto = folium.Map(location=[43.72,-79.38], zoom_start =10.75)
for i in range(0,len(latitude)):
    folium.Marker([latitude[i],longitude[i]]).add_to(maps_toronto)
# folium.Marker([latitude[5],longitude[5]], popup = 'Rouge, Malvern').add_to(maps_toronto)

maps_toronto

In [180]:
%%html
<img src="new_img.JPG", width=800,height=800>